<a href="https://colab.research.google.com/github/OseiasBeu/Cardiovascular-diseases-risk-project/blob/main/ML_Cardiovascular_diseases_risk_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libs

In [52]:
#Leitura do dataset
import requests
import zipfile
import io
import os
import numpy as np
import pandas as pd

#Modelos
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier

#Métricas
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# Exportação do Modelo
import joblib

## Carregamento dos datasets

In [2]:
# URL do arquivo ZIP
url = "https://github.com/OseiasBeu/Cardiovascular-diseases-risk-project/raw/main/dataset.zip"

# Diretório onde deseja extrair o conteúdo do arquivo zip
diretorio_destino = '/content/dataset/'

# Faz o download do arquivo ZIP
response = requests.get(url)
zip_file = io.BytesIO(response.content)

# Verifica se o diretório de destino existe, senão cria
if not os.path.exists(diretorio_destino):
    os.makedirs(diretorio_destino)

# Descompacta o arquivo zip
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(diretorio_destino)


In [8]:
X_train_r = pd.read_csv('/content/dataset/dataset/X_train_r_cdr.csv',sep=';')
y_train_r = pd.read_csv('/content/dataset/dataset/y_train_r_cdr.csv',sep=';')

X_test_r = pd.read_csv('/content/dataset/dataset/X_test_r_cdr.csv',sep=';')
y_test_r = pd.read_csv('/content/dataset/dataset/y_teste_r_cdr.csv',sep=';')

X_train = pd.read_csv('/content/dataset/dataset/X_train_cdr.csv',sep=';')
y_train = pd.read_csv('/content/dataset/dataset/y_train_cdr.csv',sep=';')

X_test = pd.read_csv('/content/dataset/dataset/X_test_cdr.csv',sep=';')
y_test = pd.read_csv('/content/dataset/dataset/y_teste_cdr.csv',sep=';')

## Linear SVC

### Dados Balanceados

In [30]:
modelo = LinearSVC()
modelo.fit(X_train_r, y_train_r['Heart_Disease'])
previsoes = modelo.predict(X_test_r)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


#### Métricas

- Acurácia

In [35]:
print('Acurácia')
taxa_de_acerto = accuracy_score(y_test_r['Heart_Disease'], previsoes)
print("Taxa de acerto %.2f" % (taxa_de_acerto * 100))

print(10*'=====')

print('Precisão')
print("Precisão: %.20f" %(precision_score(y_test_r['Heart_Disease'], previsoes)))

print(10*'=====')
print('Recall')
print(recall_score(y_test_r['Heart_Disease'], previsoes))

print(10*'=====')
print('Matriz de Confusão:')
print(confusion_matrix(y_test_r['Heart_Disease'], previsoes))

Acurácia
Taxa de acerto 49.90
Precisão
Precisão: 0.00000000000000000000
Recall
0.0
Matriz de Confusão:
[[45329     0]
 [45515     0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


-

### Dados não balanceados

In [36]:
modelo = LinearSVC()
modelo.fit(X_train, y_train['Heart_Disease'])
previsoes = modelo.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


#### Métricas

In [38]:
print('Acurácia')
taxa_de_acerto = accuracy_score(y_test['Heart_Disease'], previsoes)
print("Taxa de acerto %.2f" % (taxa_de_acerto * 100))

print(10*'=====')

print('Precisão')
print("Precisão: %.2f" %(precision_score(y_test['Heart_Disease'], previsoes)))

print(10*'=====')
print('Recall')
print(recall_score(y_test['Heart_Disease'], previsoes))

print(10*'=====')
print('Matriz de Confusão:')
print(confusion_matrix(y_test['Heart_Disease'], previsoes))

Acurácia
Taxa de acerto 91.90
Precisão
Precisão: 0.48
Recall
0.010006003602161296
Matriz de Confusão:
[[56719    55]
 [ 4947    50]]


## BernoulliNB

In [39]:
bnb = BernoulliNB()

### Dados Balanceados

In [40]:
bnb.fit(X_train_r, y_train_r['Heart_Disease'])
predito_BNb = bnb.predict(X_test_r)

#### Métricas

In [41]:
print('Acurácia')
taxa_de_acerto = accuracy_score(y_test_r['Heart_Disease'], predito_BNb)
print("Taxa de acerto %.2f" % (taxa_de_acerto * 100))

print(10*'=====')

print('Precisão')
print("Precisão: %.20f" %(precision_score(y_test_r['Heart_Disease'], predito_BNb)))

print(10*'=====')
print('Recall')
print(recall_score(y_test_r['Heart_Disease'], predito_BNb))

print(10*'=====')
print('Matriz de Confusão:')
print(confusion_matrix(y_test_r['Heart_Disease'], predito_BNb))

Acurácia
Taxa de acerto 74.26
Precisão
Precisão: 0.73663836430908735675
Recall
0.756739536416566
Matriz de Confusão:
[[33015 12314]
 [11072 34443]]


### Não Balanceados

In [42]:
#treinar o modelo
bnb.fit(X_train, y_train['Heart_Disease'])
predito_BNb = bnb.predict(X_test)

taxa_de_acerto = accuracy_score(y_test['Heart_Disease'], predito_BNb)
print("Taxa de acerto %.2f" % (taxa_de_acerto * 100))

Taxa de acerto 86.68


#### Métricas

In [43]:
print('Acurácia')
taxa_de_acerto = accuracy_score(y_test['Heart_Disease'], predito_BNb)
print("Taxa de acerto %.2f" % (taxa_de_acerto * 100))

print(10*'=====')

print('Precisão')
print("Precisão: %.20f" %(precision_score(y_test['Heart_Disease'], predito_BNb)))

print(10*'=====')
print('Recall')
print(recall_score(y_test['Heart_Disease'], predito_BNb))

print(10*'=====')
print('Matriz de Confusão:')
print(confusion_matrix(y_test['Heart_Disease'], predito_BNb))

Acurácia
Taxa de acerto 86.68
Precisão
Precisão: 0.28156258448229248836
Recall
0.4168501100660396
Matriz de Confusão:
[[51459  5315]
 [ 2914  2083]]


## DTC

In [44]:
dtc = DecisionTreeClassifier(criterion='entropy', random_state=42)

### Dados Balanceados

In [49]:
dtc.fit(X_train_r, y_train_r['Heart_Disease'])
predito_ArvoreDecisao_r = dtc.predict(X_test_r)

#### Métricas

In [50]:
print('Acurácia')
taxa_de_acerto = accuracy_score(y_test_r['Heart_Disease'], predito_ArvoreDecisao_r)
print("Taxa de acerto %.2f" % (taxa_de_acerto * 100))

print(10*'=====')

print('Precisão')
print("Precisão: %.20f" %(precision_score(y_test_r['Heart_Disease'], predito_ArvoreDecisao_r)))

print(10*'=====')
print('Recall')
print(recall_score(y_test_r['Heart_Disease'], predito_ArvoreDecisao_r))

print(10*'=====')
print('Matriz de Confusão:')
print(confusion_matrix(y_test_r['Heart_Disease'], predito_ArvoreDecisao_r))

Acurácia
Taxa de acerto 92.69
Precisão
Precisão: 0.92355133262143962458
Recall
0.9310996374821487
Matriz de Confusão:
[[41821  3508]
 [ 3136 42379]]


### Não balanceados

In [47]:
dtc.fit(X_train, y_train['Heart_Disease'])
predito_ArvoreDecisao = dtc.predict(X_test)

#### Métricas

In [48]:
print('Acurácia')
taxa_de_acerto = accuracy_score(y_test['Heart_Disease'], predito_ArvoreDecisao)
print("Taxa de acerto %.2f" % (taxa_de_acerto * 100))

print(10*'=====')

print('Precisão')
print("Precisão: %.20f" %(precision_score(y_test['Heart_Disease'], predito_ArvoreDecisao)))

print(10*'=====')
print('Recall')
print(recall_score(y_test['Heart_Disease'], predito_ArvoreDecisao))

print(10*'=====')
print('Matriz de Confusão:')
print(confusion_matrix(y_test['Heart_Disease'], predito_ArvoreDecisao))

Acurácia
Taxa de acerto 86.77
Precisão
Precisão: 0.21003649635036497623
Recall
0.23033820292175305
Matriz de Confusão:
[[52445  4329]
 [ 3846  1151]]


## Escolha do modelos

1. **Acurácia:** Percentual e vezes em que o modelo acertou. Embora seja fácil de entender, a acurácia pode não ser a melhor métrica em todos os cenários, especialmente quando as classes estão desbalanceadas.

2. **Recall (Sensibilidade ou Taxa de Verdadeiros Positivos):** O recall mede a proporção de exemplos positivos reais que foram corretamente classificados como positivos pelo modelo. É sobre pegar tudo o que é positivo. Quanto do "positivo" real o modelo capturou?

3. **Precisão:** Em outras palavras, é a capacidade do modelo de não classificar erroneamente exemplos negativos como positivos. É sobre não se enganar quando diz que é positivo. Quanto do que o modelo diz ser positivo realmente é positivo? Exemplo: Dizer que uma moeda é justa e ela realmente ser justa.

4. **Matriz de Confusão:** A matriz de confusão é uma tabela que apresenta um resumo das classificações feitas pelo modelo, comparando essas classificações com os resultados reais. Ela é especialmente útil para entender o desempenho do modelo em diferentes classes. A matriz de confusão é composta por quatro valores:
  - Verdadeiros Positivos (VP): Exemplos positivos corretamente classificados como positivos.
  - Verdadeiros Negativos (VN): Exemplos negativos corretamente classificados como negativos.
  - Falsos Positivos (FP): Exemplos negativos erroneamente classificados como positivos.
  - Falsos Negativos (FN): Exemplos positivos erroneamente classificados como negativos.

Em resumo:
Pense em um teste de bola de futebol:
- **Acurácia:** Quantas vezes o jogador acertou os chutes?
- **Recall:** Quantas vezes o jogador pegou a bola quando ela estava vindo em sua direção?
- **Precisão:** Quantas vezes o jogador disse que ia marcar um gol e realmente marcou?
- **Matriz de Confusão:** Quantas vezes o jogador achou que a bola iria para a esquerda, mas foi para a direita, e vice-versa?

In [51]:
print('Acurácia')
taxa_de_acerto = accuracy_score(y_test_r['Heart_Disease'], predito_ArvoreDecisao_r)
print("Taxa de acerto %.2f" % (taxa_de_acerto * 100))

print(10*'=====')

print('Precisão')
print("Precisão: %.20f" %(precision_score(y_test_r['Heart_Disease'], predito_ArvoreDecisao_r)))

print(10*'=====')
print('Recall')
print(recall_score(y_test_r['Heart_Disease'], predito_ArvoreDecisao_r))

print(10*'=====')
print('Matriz de Confusão:')
print(confusion_matrix(y_test_r['Heart_Disease'], predito_ArvoreDecisao_r))

Acurácia
Taxa de acerto 92.69
Precisão
Precisão: 0.92355133262143962458
Recall
0.9310996374821487
Matriz de Confusão:
[[41821  3508]
 [ 3136 42379]]


  - Tendo como base as métricas listadas e explicadas anteriosmente, o melhor modelo para este caso é o modelo de `Árvore de decisão` com `dados balanceados`

## Exportando o modelo

In [53]:
filename = 'dtc_model_cdr_r.pkl'

# Exportar o modelo para o arquivo .pkl
joblib.dump(dtc, filename)

print(f"Modelo de árvore de decisão exportado para {filename}")

Modelo de árvore de decisão exportado para dtc_model_cdr_r.pkl
